In [166]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [167]:
wiki = requests.get(url).text
soup = BeautifulSoup(wiki, 'lxml')
# print(soup.prettify())

In [168]:
table = soup.find("table", attrs={"class": "wikitable sortable"})
header_data = table.tbody.find_all("tr")[0]
table_data = table.tbody.find_all("td")
postcode=[]
borough=[]
neighbourhood=[]

In [169]:
# Get all the headings of Lists
headings = []
for td in header_data.find_all("th"):

    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip().replace(' ', '_'))

print(headings)

['Postal_Code', 'Borough', 'Neighbourhood']


In [170]:
for k in range(0,540,3):
    postcode.append(table_data[k].text.replace('\n', '').strip())
    borough.append(table_data[k+1].text.replace('\n', '').strip())
    neighbourhood.append(table_data[k+2].text.replace('\n', '').strip())

In [171]:
df = pd.DataFrame(zip(postcode,borough,neighbourhood),columns=headings)
df.head()

,Postal_Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal_Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB
